In [ ]:
import lsst.daf.persistence as dafPersist
import lsst.analysis.deblender as deblender
import lsst.afw.display as afwDisplay

disp1  = afwDisplay.Display(1)
disp2 = afwDisplay.Display(2)

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (13, 8)

In [ ]:
dataPath = "/Volumes/RHLData/hsc-v13_0"

butler = dafPersist.Butler(dataPath)
skymap = butler.get("deepCoadd_skyMap")

### Read some data

In [ ]:
if True:
    dataId = dict(tract=8766, patch='2,1', filter='HSC-I')
else:
    dataId = dict(tract=9348, patch='7,6', filter='HSC-I')

coadd = butler.get("deepCoadd_calexp", dataId, immediate=True)

ss = butler.get('deepCoadd_meas', dataId)
families = deblender.Families(ss, butler, nChildMin=0)

nameNchild = "deblend_nChild"                # Old HSC v. LSST convention
if nameNchild not in ss.getSchema().extract("*").keys():
    nameNchild = "deblend_nchild"

assert ss.getSchema().find(nameNchild) is not None

### Display it

In [ ]:
disp1.mtv(coadd, title="%(tract)s %(patch)s" % dataId)

with disp1.Buffering():
    for s in ss:
        disp1.dot('+' if s[nameNchild] > 0 else 'x',
                  *s.getCentroid(), size=3, ctype=afwDisplay.RED)

### You only need to set these dicts to make 'r' work as a callback to show RGB images of blends

In [ ]:
deblender = reload(deblender)
for bandName in "GRI":
    filterName = "HSC-%s" % bandName.upper()
    
    deblender.coaddDict[filterName] = butler.get("deepCoadd_calexp", dataId, filter=filterName)
    sources = butler.get('deepCoadd_meas', dataId, filter=filterName)
    deblender.familiesDict[filterName] = deblender.Families(sources, butler, nChildMin=0)

### Show the deblended families

Put us into a loop waiting on the display.  Sigh.

Live keys in display window:
-  d on an object in frame frame to show the children in display
-  l  Toggle various useful stretches for the children
-  r  Display RGB mosaic of children in notebook (you must have run previous cell)
-  1 2 3 4  Zoom levels for children
-  q to quit the interactive loop

In [ ]:
deblender.showBlend(coadd, families, display=disp2, imageDisplay=disp1)

### Here's how to take ra, dec and get (tract, patch)

In [ ]:
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord

def decimalRaDec(raStr, decStr):
    """Convert ra, dec as strings to decimals.
    A leading sign for the dec field is optional

    E.g. decimalRaDec("09:59:21.76", "+02:06:38.4")
    """
    ra, dec = raStr.split(':'), decStr.split(':')
    if dec[0][0] in "-+":
        decSign = +1 if dec[0][0] == '+' else -1
        dec[0] = dec[0][1:]
    else:
        decSign = +1

    d, m, s = [float(_) for _ in dec]
    dec = decSign*(d + (m + s/60.0)/60.0)

    h, m, s = [float(_) for _ in ra]
    ra = 15*(h + (m + s/60.0)/60.0)

    return ra, dec

ra, dec = "02:24:16.42", "-04:16:41.88"
ra, dec = decimalRaDec(ra, dec)
    
raDec = afwCoord.Coord(ra*afwGeom.degrees, dec*afwGeom.degrees)
tractInfo, patchInfo = skymap.findClosestTractPatchList([raDec])[0]

dataId = dict(tract = tractInfo.getId(),
              patch = "%d,%d" % patchInfo[0].getIndex(),
              filter='HSC-Z')

print dataId["tract"], dataId["patch"]

### Playground